## Final Capstone project - Applied Data Science Capstone

The objective of this notebook is to provide some support when buying a new house in Santander, a city from the north of Spain. To do so, this notebook will provide a clear view of the existing districts in the city, and the similarity among them. Therefore, the user that is planning to buy a house can know easily whether the different districts have similar services or not (e.g. number of hospitals, bus stops, etc.).

The different steps to perform such goal are the following:
1. Obtain the geographical data from Santander (polygons in which the city is divided) and create the dataframe.
2. Format the data into a usable dataframe (e.g. coordinates, remove unneeded data fields, etc.).
3. Draw the map depending on the inhabitants in each district.
4. Get information from foursquare about each district: hospitals, schools, etc.
5. Include this information normalised into the dataframe.
6. Use Kmeans to divide the districts depending on their similarity.
7. Draw a map to show similarity among districts.

In [1]:
client_id_fs = ''
secret_id_fs = ''

In [2]:
import numpy as np
import pandas as pd
import json # library to handle JSON files
import requests
import math

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

### As a first step, we obtain data from the open data platform datos.santander.es
This step is to get the different sections in which the city is divided, and format the data in a dataframe, to be used later on in with folium and the foursquare API.

In [3]:
sections_url = 'http://datos.santander.es/api/rest/datasets/secciones.csv?items=148&rnd=1880800499'
sections_csv = requests.get(sections_url).text

df = pd.read_csv(sections_url)

In [4]:
df.head(1)

,dc:identifier,dc:modified,ayto:distrito,ayto:seccion,ayto:desSec,ayto:poblacion,ayto:fecha,dct:spatial,uri
0,74,2020-02-27T02:00:01.721Z,5,8,5008,888.0,2017-01-03T01:00:01.000Z,"POLYGON (( 433925.21690000 4811910.92160000\,...",http://datos.santander.es/api/datos/secciones/...


In [5]:
df.describe()

,dc:identifier,ayto:distrito,ayto:seccion,ayto:desSec,ayto:poblacion
count,148.000000,148.000000,148.000000,148.000000,147.000000
mean,77.824324,4.851351,10.844595,4862.195946,1182.503401
std,50.843593,2.414144,7.104385,2415.443948,381.943576
min,1.000000,1.000000,1.000000,1001.000000,571.000000
25%,38.750000,2.000000,5.000000,2026.750000,940.500000
50%,75.500000,5.000000,10.000000,5007.500000,1116.000000
75%,112.250000,7.000000,15.000000,7015.250000,1384.500000
max,322.000000,8.000000,29.000000,8029.000000,2372.000000


In [6]:
df = df.drop("dc:modified", axis=1)
df = df.drop("ayto:fecha", axis=1)
df = df.drop("uri",axis=1)
df = df.drop("ayto:desSec",axis=1)
df = df.drop("ayto:distrito",axis=1)
df = df.drop("ayto:seccion",axis=1)
df.set_index('dc:identifier')
df.head(1)

,dc:identifier,ayto:poblacion,dct:spatial
0,74,888.0,"POLYGON (( 433925.21690000 4811910.92160000\,..."


### Formating spatial data to coordinates
Now we need to format the dct:spatial column to get the data in lat / long for the foursquare dataset.
First, we create a conversion function to process each data pair. Note that there are multiple data pairs in each cell as they are polygons. Additionally, we also create another column with the json format ready to be represented using folium in a map.

In [7]:
# We create a conversion function.
def utmToLatLng(zone, easting, northing, northernHemisphere=True):
    if not northernHemisphere:
        northing = 10000000 - northing

    a = 6378137
    e = 0.081819191
    e1sq = 0.006739497
    k0 = 0.9996

    arc = northing / k0
    mu = arc / (a * (1 - math.pow(e, 2) / 4.0 - 3 * math.pow(e, 4) / 64.0 - 5 * math.pow(e, 6) / 256.0))

    ei = (1 - math.pow((1 - e * e), (1 / 2.0))) / (1 + math.pow((1 - e * e), (1 / 2.0)))

    ca = 3 * ei / 2 - 27 * math.pow(ei, 3) / 32.0

    cb = 21 * math.pow(ei, 2) / 16 - 55 * math.pow(ei, 4) / 32
    cc = 151 * math.pow(ei, 3) / 96
    cd = 1097 * math.pow(ei, 4) / 512
    phi1 = mu + ca * math.sin(2 * mu) + cb * math.sin(4 * mu) + cc * math.sin(6 * mu) + cd * math.sin(8 * mu)

    n0 = a / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (1 / 2.0))

    r0 = a * (1 - e * e) / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (3 / 2.0))
    fact1 = n0 * math.tan(phi1) / r0

    _a1 = 500000 - easting
    dd0 = _a1 / (n0 * k0)
    fact2 = dd0 * dd0 / 2

    t0 = math.pow(math.tan(phi1), 2)
    Q0 = e1sq * math.pow(math.cos(phi1), 2)
    fact3 = (5 + 3 * t0 + 10 * Q0 - 4 * Q0 * Q0 - 9 * e1sq) * math.pow(dd0, 4) / 24

    fact4 = (61 + 90 * t0 + 298 * Q0 + 45 * t0 * t0 - 252 * e1sq - 3 * Q0 * Q0) * math.pow(dd0, 6) / 720

    lof1 = _a1 / (n0 * k0)
    lof2 = (1 + 2 * t0 + Q0) * math.pow(dd0, 3) / 6.0
    lof3 = (5 - 2 * Q0 + 28 * t0 - 3 * math.pow(Q0, 2) + 8 * e1sq + 24 * math.pow(t0, 2)) * math.pow(dd0, 5) / 120
    _a2 = (lof1 - lof2 + lof3) / math.cos(phi1)
    _a3 = _a2 * 180 / math.pi

    latitude = 180 * (phi1 - fact1 * (fact2 + fact3 + fact4)) / math.pi

    if not northernHemisphere:
        latitude = -latitude

    longitude = ((zone > 0) and (6 * zone - 183.0) or 3.0) - _a3
    
    longitude = longitude - 0.0013
    latitude = latitude - 0.0018
    
    return (round(latitude, 4), round(longitude, 4))

In [8]:
print(utmToLatLng(30, 433925.21690000, 4811910.9216))

(43.4554, -3.818)


In [9]:
def clean_coords(row):
    x = row['dct:spatial']
    x = x[12:-2]
    x = x.replace('\, ', ';')
    x = x.replace(' ', ',')
    x = x.split(';')
    x_new = ''
    for i in range(len(x)):
        utm = x[i].split(',')
        lat_utm = float(utm[0])
        long_utm = float(utm[1])
        final_coords = utmToLatLng(30, lat_utm, long_utm)
        x_new = x_new + str(final_coords[0]) + ',' + str(final_coords[1]) + ';'
    return x_new[:-1]

def get_json_from_clean_coords(row):
    x = row['polygon']
    x = x.split(';')
    geo_json = {"geometry": {
                "coordinates": [],
                "type": "Polygon",
            },
            "properties": {
                "stroke": "#fc1717",
                "stroke-opacity": 1,
                "stroke-width": 2
            },
            "type": "Feature"
        }
    coordinates_list = []
    for i in range(len(x)):
        coords = x[i].split(',')
        coordinates_list.append([float(coords[1]),float(coords[0])])
    
    geo_json['geometry']['coordinates'] = [coordinates_list]
    return json.dumps(geo_json)

df['polygon']  = df.apply(clean_coords,axis=1)
df['geo_json'] = df.apply(get_json_from_clean_coords,axis=1)
df.head(5)

,dc:identifier,ayto:poblacion,dct:spatial,polygon,geo_json
0,74,888.0,"POLYGON (( 433925.21690000 4811910.92160000\,...","43.4554,-3.818;43.4555,-3.8179;43.4556,-3.8177...","{""geometry"": {""coordinates"": [[[-3.818, 43.455..."
1,71,850.0,"POLYGON (( 433554.69320000 4811653.46650000\,...","43.453,-3.8226;43.4533,-3.8215;43.4543,-3.822;...","{""geometry"": {""coordinates"": [[[-3.8226, 43.45..."
2,73,696.0,"POLYGON (( 433967.47030000 4811801.76210000\,...","43.4544,-3.8175;43.4544,-3.8176;43.4542,-3.818...","{""geometry"": {""coordinates"": [[[-3.8175, 43.45..."
3,65,1241.0,"POLYGON (( 434247.72760000 4811903.68990000\,...","43.4554,-3.814;43.457,-3.8148;43.4567,-3.8158;...","{""geometry"": {""coordinates"": [[[-3.814, 43.455..."
4,68,1266.0,"POLYGON (( 433967.47030000 4811801.76210000\,...","43.4544,-3.8175;43.4544,-3.8174;43.4544,-3.817...","{""geometry"": {""coordinates"": [[[-3.8175, 43.45..."


### Now we print the map with folium to see the districts

In [10]:
latitude = 43.457
longitude = -3.816
geoJsonData = {
    "features": [
    ],
    "type": "FeatureCollection"
}
for i, row in df.iterrows():
    geoJsonData['features'].append(json.loads(row['geo_json']))

m = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.GeoJson(geoJsonData,
    style_function=lambda x: {
        'color' : x['properties']['stroke'],
        'weight' : x['properties']['stroke-width'],
        'opacity': 0.6,
        'fillColor' : x['properties']['stroke'],
        }).add_to(m)
m

In [11]:
latitude = 43.457
longitude = -3.816
geoJsonData = {
    "features": [
    ],
    "type": "FeatureCollection"
}

for i, row in df.iterrows():
    geoJsonData['features'].append(json.loads(row['geo_json']))
    geoJsonData['features'][i]['id'] = row['dc:identifier']
    
m = folium.Map(location=[latitude, longitude], zoom_start=12)

m.choropleth(
    geo_data=geoJsonData,
    name='choropleth',
    data=df,
    columns=['dc:identifier', 'ayto:poblacion'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population'
)


m

### Obtaining the additional data from foursquare API and adding it to the dataframe
Now we will use the polygon data to gather information from existing venues in the area, so we can group the districts depending on the venues per population. For example, number of hospitals per inhabitant.

In [12]:
categories = {
    'hospital' : '4bf58dd8d48988d104941735',
    'restaurant' : '4d4b7105d754a06374d81259',
    'school' : '4bf58dd8d48988d13b941735',
    'bus_stop' : '52f2ab2ebcbc57f1066b8b4f'
}

foursquare_api_endpoint = 'https://api.foursquare.com/v2/venues/explore?' + 'client_secret=' + secret_id_fs + '&client_id=' + client_id_fs

Testing with the first district to obtain hospitals

In [13]:
fs_final = foursquare_api_endpoint + '&v=20200201' + '&categoryId=' + categories['hospital'] + '&polygon=' + df['polygon'][0]
res = requests.get(fs_final).text
print(res)

{"meta":{"code":200,"requestId":"5e579a8183525f001cba3191"},"response":{"headerLocation":"Santander","headerFullLocation":"Santander","headerLocationGranularity":"city","query":"medical","totalResults":4,"suggestedBounds":{"ne":{"lat":43.4598,"lng":-3.8102},"sw":{"lat":43.4342,"lng":-3.8397}},"groups":[{"type":"Recommended Places","name":"recommended","items":[{"reasons":{"count":0,"items":[{"summary":"This spot is popular","type":"general","reasonName":"globalInteractionReason"}]},"venue":{"id":"4c0e659db1b676b0e5f6e186","name":"Valdecilla Sur","location":{"lat":43.45504087562055,"lng":-3.8296120539157057,"labeledLatLngs":[{"label":"display","lat":43.45504087562055,"lng":-3.8296120539157057}],"postalCode":"39011","cc":"ES","city":"Santander","state":"Cantabria","country":"España","formattedAddress":["39011 Santander Cantabria","España"]},"categories":[{"id":"4bf58dd8d48988d177941735","name":"Doctor's Office","pluralName":"Doctor's Offices","shortName":"Doctor's Office","icon":{"prefix

We obtain the final dataframe by including the number of different types of venues per number of people living in the area.

In [14]:
df_data = df
max_chars = 2048

def get_venues(polygon, population, keyn):
    x_new = ''
    if len(polygon) > max_chars:
        d = round(len(polygon) / max_chars)
        x = polygon.split(';')
        l = 0
        while l < len(x):
            x_new = x_new + x[l] + ';'
            l = l + d
        x_new = x_new + x[-1]
    else:
        x_new = polygon
    fs_url = foursquare_api_endpoint + '&v=20200201' + '&categoryId=' + categories[keyn] + '&polygon=' + x_new
    
    try:
        json_object = requests.get(fs_url).json()
    except:
        print(len(fs_url))
        print(fs_url)
    # print(json_object['response'])
    try:
        res = json_object['response']['totalResults'] / population
    except:
        print(json_object)
        res = 0
    return res

for key in categories.keys():
    df_data[key]  = df_data.apply(lambda x: get_venues(x['polygon'], x['ayto:poblacion'], key), axis=1)
    df_data[key + '_norm'] = (df_data[key]-df_data[key].min())/(df_data[key].max()-df_data[key].min())

#df_data['hospital']  = df_data.apply(lambda x: get_venues(x['polygon'], x['ayto:poblacion'], 'hospital'), axis=1)
#df_data['hospital' + '_norm'] = (df_data['hospital']-df_data['hospital'].min())/(df_data['hospital'].max()-df_data['hospital'].min())

df_data.head()

,dc:identifier,ayto:poblacion,dct:spatial,polygon,geo_json,hospital,hospital_norm,restaurant,restaurant_norm,school,school_norm,bus_stop,bus_stop_norm
0,74,888.0,"POLYGON (( 433925.21690000 4811910.92160000\,...","43.4554,-3.818;43.4555,-3.8179;43.4556,-3.8177...","{""geometry"": {""coordinates"": [[[-3.818, 43.455...",0.004505,0.142080,0.019144,0.599410,0.004505,0.857357,0.0,0.0
1,71,850.0,"POLYGON (( 433554.69320000 4811653.46650000\,...","43.453,-3.8226;43.4533,-3.8215;43.4543,-3.822;...","{""geometry"": {""coordinates"": [[[-3.8226, 43.45...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,73,696.0,"POLYGON (( 433967.47030000 4811801.76210000\,...","43.4544,-3.8175;43.4544,-3.8176;43.4542,-3.818...","{""geometry"": {""coordinates"": [[[-3.8175, 43.45...",0.001437,0.045318,0.012931,0.404875,0.001437,0.273467,0.0,0.0
3,65,1241.0,"POLYGON (( 434247.72760000 4811903.68990000\,...","43.4554,-3.814;43.457,-3.8148;43.4567,-3.8158;...","{""geometry"": {""coordinates"": [[[-3.814, 43.455...",0.002417,0.076249,0.003223,0.100920,0.000000,0.000000,0.0,0.0
4,68,1266.0,"POLYGON (( 433967.47030000 4811801.76210000\,...","43.4544,-3.8175;43.4544,-3.8174;43.4544,-3.817...","{""geometry"": {""coordinates"": [[[-3.8175, 43.45...",0.000790,0.024914,0.003160,0.098927,0.000790,0.150342,0.0,0.0


### Use K-means to obtain similar districts in the city of Santander
To do so, we use the K-means algorithm to find them.

In [16]:
# Prepare de dataframe to fit the kmeans
df_kmeans = df_data
df_kmeans = df_kmeans.drop('dct:spatial', axis=1)
df_kmeans = df_kmeans.drop('ayto:poblacion', axis=1)
df_kmeans = df_kmeans.drop('polygon', axis=1)
df_kmeans = df_kmeans.drop('geo_json', axis=1)
df_kmeans = df_kmeans.drop('hospital', axis=1)
df_kmeans = df_kmeans.drop('restaurant', axis=1)
df_kmeans = df_kmeans.drop('school', axis=1)
df_kmeans = df_kmeans.drop('bus_stop', axis=1)

df_kmeans = df_kmeans.fillna(0)


In [17]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_kmeans)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
df_kmeans.insert(0, 'Cluster', kmeans.labels_)

### Finally, we draw a map representing the similar districts in the city
This map is created using the groups provided by the K-means algorithm.

In [20]:
df_final = df_kmeans
df_final.insert(0, 'geo_json', df_data['geo_json'])
df_final.head()

,geo_json,Cluster,dc:identifier,hospital_norm,restaurant_norm,school_norm,bus_stop_norm
0,"{""geometry"": {""coordinates"": [[[-3.818, 43.455...",4,74,0.142080,0.599410,0.857357,0.0
1,"{""geometry"": {""coordinates"": [[[-3.8226, 43.45...",4,71,0.000000,0.000000,0.000000,0.0
2,"{""geometry"": {""coordinates"": [[[-3.8175, 43.45...",4,73,0.045318,0.404875,0.273467,0.0
3,"{""geometry"": {""coordinates"": [[[-3.814, 43.455...",4,65,0.076249,0.100920,0.000000,0.0
4,"{""geometry"": {""coordinates"": [[[-3.8175, 43.45...",4,68,0.024914,0.098927,0.150342,0.0


In [21]:
latitude = 43.457
longitude = -3.816
geoJsonData = {
    "features": [
    ],
    "type": "FeatureCollection"
}

for i, row in df_final.iterrows():
    geoJsonData['features'].append(json.loads(row['geo_json']))
    geoJsonData['features'][i]['id'] = row['dc:identifier']
    
m = folium.Map(location=[latitude, longitude], zoom_start=12)

m.choropleth(
    geo_data=geoJsonData,
    name='choropleth',
    data=df_final,
    columns=['dc:identifier', 'Cluster'],
    key_on='feature.id',
    fill_color='RdYlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Clusters'
)


m

In case the folium map does not load properly in github:
![alt text](https://github.com/juanrasantana/Coursera_Capstone/raw/master/santander_city_sections_similarity.png "Choropleth map of Santander city sections showing their similarity")